Demand Estimation: Problem Set 2

Moonju Cho, Max Schnidman, Yen Ling Tan, Yang Yu

Q1: Deriving market shares as a function of $\delta$ and inverting to get $\delta$ as a function of market shares

Consumers have the following utility function:
\begin{align}
u_{ijt} &= \delta_{jt} + \epsilon_{ijt}\\
\delta_{jt} &= \beta x_{jt} -\alpha p_{jt} +\xi_{jt}
\end{align}

where $x_{jt}$ is a vector of product characteristics, $p_{jt}$ is the price of product $j$ at time $t$, and $\xi_{jt}$ is a vector of unobserved characteristics. The $\epsilon_{ijt}$ are i.i.d. extreme value type I.

By i.i.d. extreme value type I, we have
\begin{align}
s_{jt}=\begin{cases}
\begin{array}{cc}
\frac{\exp\left(\delta_{jt}\right)}{1+\sum_{k=1}^{J_{t}}\exp\left(\delta_{kt}\right)} & j\geq 1\\
\frac{1}{1+\sum_{k=1}^{J_{t}}\exp\left(\delta_{kt}\right)} & j=0
\end{array}\end{cases}
\end{align}

Taking logs and subtracting $s_{0t}$ from $s_{jt}$, we have
\begin{align}
\ln s_{jt}-\ln s_{0t}=\delta_{jt}=x_{jt}\beta-\alpha p_{jt}+\xi_{jt}
\end{align}

$s_{jt}$ comes from the data and is equal to $\frac{q_{jt}}{M_t}$, making the share of the outside good $s_{0t}=1-\sum_{j=1}^{J_t}s_{jt}$

Q2: 

Under pooling, We call product $j$ in year $t$ product $l(j,t)$, so that product $l(j,t)$ is different than product $l(j,t+1)$.

Under pooling, our utility function would be:

\begin{align}
u_{il} &= \delta_{l} + \epsilon_{il}\\
\delta_{l} &= \beta x_{l} -\alpha p_{l} +\xi_{l}
\end{align}

We still have
\begin{align}
s_{l}=\begin{cases}
\begin{array}{cc}
\frac{\exp\left(\delta_{l}\right)}{1+\sum_{k=1}^{L}\exp\left(\delta_{k}\right)} & l\geq 1\\
\frac{1}{1+\sum_{k=1}^{L}\exp\left(\delta_{k}\right)} & l=0
\end{array}\end{cases}
\end{align}


By the same logic as in Q1, 
\begin{align}
\ln s_{l}-\ln s_{0}=\delta_{l}=x_{l}\beta-\alpha p_{l}+\xi_{l}
\end{align}

$s_{l}$ comes from data and is euqal to $\frac{q_{l}}{M}$, making the share of the outside good $s_{0}=1-\sum_{l=1}^{L}s_{l}$

The key difference is that the outside good share becomes lower under pooling if the total market size does not increase. This occurs because consumers can now substitute both
across products and across time (i.e., a consumer can choose to purchase the same car in 1990 or 1991, and these are separate products in the data.)

Q3: Estimating the demand system under logit assumptions

First, we import the necessary packages and data.

In [28]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.sandbox.regression.gmm as gmm
import os

In [29]:
# Read in data
# parent_dir = os.path.dirname(os.getcwd())
# CarData = pd.read_csv(parent_dir + '\\data\\data_ps2_3nests.txt', sep='\t', header=None)
CarData = pd.read_csv("C:\\Users\\chv7bg\\Documents\\coursework\\empirical-io\\Mortimer HW\\data\\ps2_data_3nests.txt", sep='\t', header=None)
#name columns
CarData.columns = ['Car_ID', 'Year', 'Firm_ID', 'Price', 'Quantity', 'Weight', 'HP', 'AC', 'Nests']
#display data
CarData.head()

,Car_ID,Year,Firm_ID,Price,Quantity,Weight,HP,AC,Nests
0,53,1990,6,18820,32410,2919,114,1,2
1,105,1990,18,9456,124135,2759,88,0,1
2,71,1990,5,13071,1276,2455,97,0,2
3,120,1990,5,11499,44906,2620,130,0,2
4,128,1990,16,6851,39602,2194,81,0,1


Now, we define the market size for each year and use it to generate market shares.

In [30]:
M = 100_000_000 #set total market size
CarData['Market_Share'] = CarData['Quantity'] / M #construct market share as the ratio of sales to total market size
CarData['Outside_Share'] = 1 - (CarData['Quantity'].groupby(CarData['Year']).transform('sum') / M )#construct outside share as 1 minus the ratio of sales in a given year to total market size

We create the instruments for price. To compute the mean characteristics of rivals, we take the difference of the sum of all product characteristics and the characteristics of the firm's products, and divide that by the number of rivals products in the market.

In [31]:
CarData['Rival_Weight'] = (CarData['Weight'].groupby(CarData['Year']).transform('sum') - CarData['Weight'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('sum'))/(CarData['Weight'].groupby(CarData['Year']).transform('count') - CarData['Weight'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('count'))
CarData['Rival_HP'] = (CarData['HP'].groupby(CarData['Year']).transform('sum') - CarData['HP'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('sum'))/(CarData['HP'].groupby(CarData['Year']).transform('count') - CarData['HP'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('count'))
CarData['Rival_AC'] = (CarData['AC'].groupby(CarData['Year']).transform('sum') - CarData['AC'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('sum'))/(CarData['AC'].groupby(CarData['Year']).transform('count') - CarData['AC'].groupby([CarData['Year'], CarData['Firm_ID']]).transform('count'))

We normalize the continuous variables by dividing by their mean.

In [32]:
CarData['Weight_Norm'] = CarData['Weight'] / CarData['Weight'].mean()
CarData['HP_Norm'] = CarData['HP'] / CarData['HP'].mean()
CarData['Price_Norm'] = CarData['Price'] / CarData['Price'].mean()

Now, we construct the GMM Estimator for the logit model. 

We create the endogenous variable, exogenous variables, and instruments.

In [33]:
Endog = np.log(CarData['Market_Share']) - np.log(CarData['Outside_Share'])
Exog = CarData[['Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC']]
Inst = CarData[['Rival_Weight', 'Rival_HP', 'Rival_AC', 'Weight_Norm', 'HP_Norm', 'AC']]
Exog = sm.add_constant(Exog)
Inst = sm.add_constant(Inst)

Using the Linear IV GMM function in statsmodels, we estimate the model. This package assumes the following form for the moment conditions:

\begin{equation*} 
E[z_{it}*(y_{it}-x_{it}\beta)]=0
\end{equation*}

Further, because the moment condition is linear, $\beta$ has the following closed form solution, which this function solves for:

\begin{equation*}
\beta_{GMM} = (X'ZWZ'X)^{-1}X'ZWZ'y
\end{equation*}

where $W$ is the weighting matrix, and $Z$ is the matrix of instruments.

In [34]:
LogitModel = gmm.LinearIVGMM(Endog, Exog, Inst) #we define the logit model, estimating using a linear IV GMM estimator
LogitModelFit = LogitModel.fit(maxiter=2).summary() #we fit the model and print the summary, setting the maximum number of iterations to 2 to implement 2-step GMM; the initial weight matrix is the identity matrix
LogitModelFit

ValueError: Unable to coerce to DataFrame, shape must be (393, 7): given (393, 1)

These results generally match our intuition: higher prices reduce the choice probability, but more powerful cars with AC increase the choice probability.

Q4. Computing elasticities. 

In [ ]:
CarData['Elast_MNL'] = -5.1050 * CarData['Price_Norm']*(1- CarData['Market_Share']) #we calculate the price elasticity of demand for each observation
CarData['Elast_MNL'].describe() #we print the summary statistics of the price elasticity of demand

count    393.000000
mean      -5.102218
std        3.622023
min      -23.084209
25%       -6.785984
50%       -3.784435
75%       -2.688700
max       -0.944018
Name: Elast_MNL, dtype: float64

In [ ]:
#sort data by quantity
CarData = CarData.sort_values(by=['Quantity'], ascending=False)

#create a 10x10 matrix of zeros
Elast_MNL = np.zeros((10,10))
#On the diagonal, we calculate the price elasticity of demand for each observation
for i in range(10):
    Elast_MNL[i,i] = -5.1050 * CarData['Price_Norm'].iloc[i]*(1- CarData['Market_Share'].iloc[i])
#On the off-diagonal, we calculate the cross-price elasticity of demand for each observation
for i in range(10):
    for j in range(10):
        if i != j:
            Elast_MNL[i,j] = 5.1050 * CarData['Price_Norm'].iloc[j]*CarData['Market_Share'].iloc[j]

#we print the matrix as a table
pd.DataFrame(Elast_MNL)


,0,1,2,3,4,5,6,7,8,9
0,-3.056766,0.015125,0.012819,0.006608,0.008651,0.010828,0.009982,0.010014,0.006414,0.006442
1,0.013452,-3.437225,0.012819,0.006608,0.008651,0.010828,0.009982,0.010014,0.006414,0.006442
2,0.013452,0.015125,-3.059927,0.006608,0.008651,0.010828,0.009982,0.010014,0.006414,0.006442
3,0.013452,0.015125,0.012819,-1.808967,0.008651,0.010828,0.009982,0.010014,0.006414,0.006442
4,0.013452,0.015125,0.012819,0.006608,-2.467249,0.010828,0.009982,0.010014,0.006414,0.006442
5,0.013452,0.015125,0.012819,0.006608,0.008651,-3.280930,0.009982,0.010014,0.006414,0.006442
6,0.013452,0.015125,0.012819,0.006608,0.008651,0.010828,-3.026347,0.010014,0.006414,0.006442
7,0.013452,0.015125,0.012819,0.006608,0.008651,0.010828,0.009982,-3.186653,0.006414,0.006442
8,0.013452,0.015125,0.012819,0.006608,0.008651,0.010828,0.009982,0.010014,-2.063580,0.006442
9,0.013452,0.015125,0.012819,0.006608,0.008651,0.010828,0.009982,0.010014,0.006414,-2.073415


The cross-price elasticity is $\alpha p_ks_k$, suggesting that the percentage increase in all other products is the same given an increase in $p_k$. It doesn't capture the possibility that the old buyers of product $k$ are more likely to switch to similar products as $k$.

The own price elasticity suggests that two products have the same markup if they have the same shares, under most pricing models.

Nested Logit Estimation

Q5. (a) In a nested logit model, consumer utility is $u_{ijt} = \delta^*_{jt}+\zeta_{igt}+(1-\sigma)\varepsilon_{ijt}$, where the group-specific and product-specific idiosyncratic taste errors ($\zeta_{igt}$ and $\varepsilon_{ijt}$) are distributed in such a way where the sum of them follows a Generalized Extreme Value distribution. $\varepsilon_{ijt}$ follows a type 1 Extreme Value distribution and $\sigma\in [0,1]$ is a parameter that governs the taste correlation between products in the same nest. 

The market share of product $j$ is divided into the group-level market share and the within-group market share. Product $j$'s within-group market share is $\mathcal{s}_{j|gt}=\frac{\exp(\delta_{jt}/(1-\sigma))}{D_{gt}}$, where $ {D}_{gt} = \sum_{k\in {J}_{gt}} \exp(\delta_{jt}/(1-\sigma) )$, and the group-level market share is $\mathcal{s}_{gt} = \frac{D_{gt}^{1-\sigma}}{\sum_{g'} D_{g't}^{1-\sigma}}$. The product's unconditional market share simplifies to

\begin{equation*}
 \mathcal{s}_{jt} = \mathcal{s}_{jt|gt}\cdot\mathcal{s}_{gt}=\frac{\exp(\delta_{jt}/(1-\sigma) )}{D_g^\sigma \sum_{g'} D_{g't}^{1-\sigma}}.
\end{equation*}

(b) When the mean utility level of the outside option is normalized as $0$, and its nest exclusively contains the outside option, its market share is $\mathcal{s}_{ot}=\frac{1}{\sum_{g'} D_{g'}^{1-\sigma}}$. Then,

\begin{equation*}
\ln s_{jt} - \ln s_{ot} = \delta_{jt}/(1-\sigma) -\sigma \ln D_{gt}=\delta_{jt}/(1-\sigma) -\frac{\sigma}{1-\sigma}\ln(s_{gt}/s_{ot})
\end{equation*}, 

 where the last equality follows from incorporating $\ln D_{gt} = \frac{\ln(s_{gt}/s_{ot})}{1-\sigma}$ from the group-level market share. By combining terms, we get the inversion: $\delta_{jt}=\ln(s_{jt}/s_{ot})-\sigma \mathcal{s}_{jt|gt}$.


(c) By moving all parameters and errors on the RHS, we can get a regression equation: $\ln(s_{jt}/s_{ot}) = x_{jt}\beta - \alpha p_{jt}+\sigma \ln s_{jt|gt} +\xi_{jt}$.

Q6. (a) The nested logit model relaxes the IIA problem in a logit model. By adding a nest-specific error term, we allow substitution patterns between products with similar characteristics to be positively correlated. The IIA problem stems from the iid additive error (iid consumer tastes), which implies that market shares and elasticities are only determined by the mean utility levels $\delta_{j}$, with no effect from individual product characteristics or prices. For example, if we have a dataset in which an inexpensive Toyota Camry has the same market share as a luxury car like Porsche 911 Carrera, then in a logit model, they have the same cross-price elasticity with any given third car in the data. This substitution pattern that is based on existing market share is unrealistic. With nested logit, products are grouped into nests so that the IIA property holds within a nest but not across nests. In other words, if we place Toyota Camry and Honda Accord in the same nest and they have the same market share, they will have the same cross-price elasticity with any given third car in the data. If we place Toyota Camry and Porsche 911 Carrera in different nests and they have the same market share, they do not need to have the same cross-price elasticity with any given third car in the data.

(b) The nested logit model has several weaknesses. First, a researcher needs to form a priori assumptions about taste correlation to group products into different nests; these assumptions can be formed without looking at the data, even though ideally we would like to let the data tell us the nests. Second, different nest structures may produce different results; the ordering of the nest structure can affect the estimates. Third, a researcher needs to be able to group the products into nests, and it may be difficult to use a nested logit model if there are no obvious nests in the product market.

Q7. (a) To estimate the nested logit model, we first create the necessary data and instruments. We create the products' within-nest market share, and take its log as an exogenous variable. We also generate instruments as the mean of rival within-nest product characteristics, considering products produced by the same firm within a given nest to be rivalrous. 

We then split the data by year to estimate three separate models.

In [ ]:
#Make a copy of the data
CarDataNL = CarData.copy()
#we calculate the market share for each observation by year and nest, then take its log
CarDataNL['Nest_Share'] = CarDataNL['Quantity']/CarDataNL['Quantity'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('sum')
CarDataNL['LogNest_Share'] = np.log(CarDataNL['Nest_Share'])
#we calculate the inside share for each observation by year, then take its log
CarDataNL['Inside_Share'] = CarDataNL['Quantity']/CarDataNL['Quantity'].groupby(CarDataNL['Year']).transform('sum')
CarDataNL['LogInside_Share'] = np.log(CarDataNL['Inside_Share'])
#we construct the instruments, this time by nest
CarDataNL['Rival_Nest_Weight'] = (CarDataNL['Weight'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('sum') - CarDataNL['Weight'])/(CarDataNL['Weight'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('count') - 1)
CarDataNL['Rival_Nest_HP'] = (CarDataNL['HP'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('sum') - CarDataNL['HP'])/(CarDataNL['HP'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('count') - 1)
CarDataNL['Rival_Nest_AC'] = (CarDataNL['AC'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('sum') - CarDataNL['AC'])/(CarDataNL['AC'].groupby([CarDataNL['Year'], CarDataNL['Nests']]).transform('count') - 1)
#next, create three dataframes, one for each year
CarDataNL_1990 = CarDataNL[CarDataNL['Year'] == 1990]
CarDataNL_1991 = CarDataNL[CarDataNL['Year'] == 1991]
CarDataNL_1992 = CarDataNL[CarDataNL['Year'] == 1992]


In [ ]:
#create endogenous and exogenous variables for each year
Endog_1990 = np.log(CarDataNL_1990['Market_Share']) - np.log(CarDataNL_1990['Outside_Share'])
Exog_1990 = CarDataNL_1990[['Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share']]
Inst_1990 = CarDataNL_1990[['Rival_Weight', 'Rival_HP', 'Rival_AC', 'Weight_Norm', 'HP_Norm', 'AC', 'Rival_Nest_Weight', 'Rival_Nest_HP', 'Rival_Nest_AC']]
Exog_1990 = sm.add_constant(Exog_1990)
Inst_1990 = sm.add_constant(Inst_1990)

Endog_1991 = np.log(CarDataNL_1991['Market_Share']) - np.log(CarDataNL_1991['Outside_Share'])
Exog_1991 = CarDataNL_1991[['Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share']]
Inst_1991 = CarDataNL_1991[['Rival_Weight', 'Rival_HP', 'Rival_AC', 'Weight_Norm', 'HP_Norm', 'AC', 'Rival_Nest_Weight', 'Rival_Nest_HP', 'Rival_Nest_AC']]
Exog_1991 = sm.add_constant(Exog_1991)
Inst_1991 = sm.add_constant(Inst_1991)

Endog_1992 = np.log(CarDataNL_1992['Market_Share']) - np.log(CarDataNL_1992['Outside_Share'])
Exog_1992 = CarDataNL_1992[['Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share']]
Inst_1992 = CarDataNL_1992[['Rival_Weight', 'Rival_HP', 'Rival_AC', 'Weight_Norm', 'HP_Norm', 'AC', 'Rival_Nest_Weight', 'Rival_Nest_HP', 'Rival_Nest_AC']]
Exog_1992 = sm.add_constant(Exog_1992)
Inst_1992 = sm.add_constant(Inst_1992)


We once again use the Linear IV GMM function in statsmodels to estimate the model for each year..

In [ ]:
#use the linear IV GMM estimator to estimate the model for each year
LogitModel_1990 = gmm.LinearIVGMM(Endog_1990, Exog_1990, Inst_1990)
LogitModelFit_1990 = LogitModel_1990.fit(maxiter=2).summary()
LogitModelFit_1990

ValueError: Unable to coerce to DataFrame, shape must be (131, 10): given (131, 1)

In [ ]:
LogitModel_1991 = gmm.LinearIVGMM(Endog_1991, Exog_1991, Inst_1991)
LogitModelFit_1991 = LogitModel_1991.fit(maxiter=2).summary()
LogitModelFit_1991

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                        12.30
Model:                    LinearIVGMM   Prob (Hansen J):                0.0152
Method:                           GMM                                         
Date:                Wed, 01 Nov 2023                                         
Time:                        09:53:30                                         
No. Observations:                 131                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.0966      0.668     -9.129      0.000      -7.406      -4.788
Price_Norm       -2.1382      0.583     -3.665      0.000      -3.282      -0.995
Weight_Norm       1.9695      0.605      3.257      0.001       0.784       3.155
HP_Norm           1.1506      0.860      1.338      0.181      -0.535       2.837
AC                0.3163      0.145      2.180      0.029       0.032       0.601
LogNest_Share     0.6283      0.081      7.795      0.000       0.470       0.786
=================================================================================
"""

In [ ]:
LogitModel_1992 = gmm.LinearIVGMM(Endog_1992, Exog_1992, Inst_1992)
LogitModelFit_1992 = LogitModel_1992.fit(maxiter=2).summary()
LogitModelFit_1992

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                        14.82
Model:                    LinearIVGMM   Prob (Hansen J):               0.00510
Method:                           GMM                                         
Date:                Wed, 01 Nov 2023                                         
Time:                        09:53:31                                         
No. Observations:                 131                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.1142      0.662     -9.234      0.000      -7.412      -4.816
Price_Norm       -1.9677      0.577     -3.413      0.001      -3.098      -0.838
Weight_Norm       2.1166      0.592      3.574      0.000       0.956       3.277
HP_Norm           1.2091      0.902      1.341      0.180      -0.558       2.976
AC                0.4307      0.137      3.154      0.002       0.163       0.698
LogNest_Share     0.6634      0.081      8.181      0.000       0.505       0.822
=================================================================================
"""

To cross-validate this, we also use a nonlinear solver to estimate the model. We use the following code to estimate the model for each year.

In [ ]:
def TwoStepGMM(Endog,Exog,Inst): #we define a function that implements the two-step GMM estimator, taking the endogenous variable, exogenous variables, and instruments as inputs
    #to determine the initial values for the nonlinear GMM estimator, we first run a linear 2SLS regression
    #For the first stage, we regress each of our endogenous variables on the instruments and calculate the fitted values
    TSLS1a = sm.OLS(Exog['Price_Norm'], Inst).fit().fittedvalues 
    TSLS1b = sm.OLS(Exog['LogNest_Share'], Inst).fit().fittedvalues
    #we then construct a dataframe with the fitted values and the exogenous variables
    ExogTSLS = pd.DataFrame({'const':np.ones(Exog.shape[0]), 'TSLS1a':TSLS1a, 'TSLS1b':TSLS1b, 'Weight_Norm':Exog['Weight_Norm'], 'HP_Norm':Exog['HP_Norm'], 'AC':Exog['AC']})
    TSLS2 = sm.OLS(Endog, ExogTSLS).fit().params #we run a second stage regression of the endogenous variable on the fitted values and the exogenous variables, and save the coefficients
    W_1 = np.identity(Inst.shape[1]) #we set the initial weight matrix to the identity matrix
    #we define a function that calculates the objective function for the nonlinear GMM estimator
    def Estimator(beta,W):
        nobs = Endog.shape[0] #we calculate the number of observations based on the number of rows in the endogenous variable
        MC = (1/nobs)*np.matmul(Inst.T, Endog - np.matmul(Exog, beta)) #we calculate the moment conditions, which as the form E[z(y-Xb)] = 0
        Obj = np.matmul(np.matmul(MC.T,W), MC) #we calculate the GMM objective function as MC'*W*MC
        return Obj
    #having defined the objective function, we now implement the first stage of the nonlinear GMM estimator
    beta_1 = sp.optimize.minimize(Estimator, x0=TSLS2, args=(W_1), method='BFGS', options={'maxiter': 1e5,'gtol': 1e-24}).x #we estimate the first-step coefficients using the BFGS algorithm, using the 2SLS coefficients as the initial values
    Resid_1 = Endog - np.matmul(Exog, beta_1) #we calculate the residuals from the first step to construct the optimal weight matrix
    W_2 = np.linalg.inv((1/Endog.shape[0]) * np.matmul(np.matmul(Inst.T, np.diag(Resid_1**2)), Inst)) #we calculate the optimal weight matrix
    beta_2 = sp.optimize.minimize(Estimator, x0=beta_1, args=(W_2), method='BFGS', options={'maxiter': 1e5,'gtol': 1e-24}).x #we estimate the second-step coefficients using the BFGS algorithm, using the first-step coefficients as the initial values and the optimal weight matrix
    Sigma = ((Endog- np.matmul(Exog,beta_2)).T @ (Endog- np.matmul(Exog,beta_2)))/(Exog.shape[0]-Exog.shape[1]) #we calculate the variance of the residuals
    #generate variance-covariance matrix
    VCOV = Sigma**2 * np.linalg.inv(np.dot(Exog.transpose(),Exog)) #we calculate the variance-covariance matrix
    #generate standard errors
    SE = np.sqrt(np.diag(VCOV)) #we calculate the standard errors
    return TSLS2 ,beta_1, beta_2, SE #this function returns the 2SLS coefficients, the first-step coefficients, the second-step coefficients, and the standard errors

Having defined our two-step GMM estimator, we now estimate the model for each year, collecting the estimates, standard errors, and T-statistics in a dataframe.

In [ ]:
beta_1990 = TwoStepGMM(Endog_1990,Exog_1990,Inst_1990)[2]
se_1990 = TwoStepGMM(Endog_1990,Exog_1990,Inst_1990)[3]
T_1990 = beta_1990/se_1990 #we calculate the t-statistics as the ratio of the coefficients to the standard errors
Res_1990 = pd.DataFrame({'Coefficient':beta_1990, 'Standard Error':se_1990, "T Statistic": T_1990}, index=['const', 'Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share'])
Res_1990

C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  W_2 = np.linalg.inv((1/Endog.shape[0]) * np.matmul(np.matmul(Inst.T, np.diag(Resid_1**2)), Inst)) #we calculate the optimal weight matrix
C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future 

,Coefficient,Standard Error,T Statistic
const,-6.461531,0.889488,-7.264322
Price_Norm,-3.279167,0.294424,-11.137551
Weight_Norm,2.126848,0.938240,2.266848
HP_Norm,1.586206,0.613275,2.586451
AC,0.587710,0.335139,1.753630
LogNest_Share,0.538611,0.075881,7.098140


In [ ]:
beta_1991 = TwoStepGMM(Endog_1991,Exog_1991,Inst_1991)[2]
se_1991 = TwoStepGMM(Endog_1991,Exog_1991,Inst_1991)[3]
T_1991 = beta_1991/se_1991
Res_1991 = pd.DataFrame({'Coefficient':beta_1991, 'Standard Error':se_1991, "T Statistic": T_1991}, index=['const', 'Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share'])
Res_1991

C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  W_2 = np.linalg.inv((1/Endog.shape[0]) * np.matmul(np.matmul(Inst.T, np.diag(Resid_1**2)), Inst)) #we calculate the optimal weight matrix
C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future 

,Coefficient,Standard Error,T Statistic
const,-7.062775,0.698353,-10.113467
Price_Norm,-2.722358,0.227830,-11.949074
Weight_Norm,2.798372,0.783250,3.572772
HP_Norm,1.583674,0.525904,3.011339
AC,0.337427,0.229078,1.472977
LogNest_Share,0.570838,0.067066,8.511623


In [ ]:
beta_1992 = TwoStepGMM(Endog_1992,Exog_1992,Inst_1992)[2]
se_1992 = TwoStepGMM(Endog_1992,Exog_1992,Inst_1992)[3]
T_1992 = beta_1992/se_1992
Res_1992 = pd.DataFrame({'Coefficient':beta_1992, 'Standard Error':se_1992, "T Statistic": T_1992}, index=['const', 'Price_Norm', 'Weight_Norm', 'HP_Norm', 'AC', 'LogNest_Share'])
Res_1992

C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  W_2 = np.linalg.inv((1/Endog.shape[0]) * np.matmul(np.matmul(Inst.T, np.diag(Resid_1**2)), Inst)) #we calculate the optimal weight matrix
C:\Users\mes7jw\AppData\Local\Temp\ipykernel_31344\3580595035.py:19: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future 

,Coefficient,Standard Error,T Statistic
const,-7.106873,0.753486,-9.431994
Price_Norm,-2.570051,0.224098,-11.468417
Weight_Norm,2.878105,0.829621,3.469179
HP_Norm,1.839356,0.561372,3.276539
AC,0.463188,0.236809,1.955953
LogNest_Share,0.616129,0.072957,8.445140


Compared to the multinomial logit model, the nested logit price coefficients are smaller, and vary slightly from year to year. This reduction in the coefficients stems in part from separating products into nests, and in part from removing intertemporal substitution as an option. The nest coefficients $\sigma$ range from $0.54$ to $0.62$, suggesting that there is a moderate amount of correlation between products in the same nest.

As before, the price coefficient is negative, and the characteristic coefficients are positive, suggesting that consumers prefer more powerful cars with air conditioning, and dislike higher prices.

(b). We can estimate a random-coefficient model using only one year of this dataset because we can leverage variation in product characteristics to estimate the parameters.

(c). The elasticities are a little trickier to calculate in the nested logit. Following Miller's notes on the nested logit http://www.nathanhmiller.org/nlnotes.pdf, we have three possible forms:

\begin{align*}
\frac{\partial s_j}{\partial p_j} &= \frac{-\alpha}{1-\sigma}s_j(1-\sigma \bar{s}_{j|g} - (1-\sigma ) s_j)\\
\frac{\partial s_j}{\partial p_k} &= \alpha s_k\left( s_j + \frac{\sigma}{1-\sigma} \bar{s}_{j|g} \right) \text{ if } j,k \in g\\
\frac{\partial s_j}{\partial p_k} &= \alpha s_k s_j \text{ if } j\in g \text{ and } k \notin g
\end{align*}

So that 
\begin{align*}
\epsilon_j &= \frac{-\alpha}{1-\sigma}p_j(1-\sigma \bar{s}_{j|g} - (1-\sigma ) s_j)\\
\epsilon_{j,k} &= \alpha \frac{s_kp_k}{s_j}\left( s_j + \frac{\sigma}{1-\sigma} \bar{s}_{j|g} \right) \text{ if } j,k \in g\\
\epsilon_{j,k} &= \alpha s_k p_k \text{ if } j\in g \text{ and } k \notin g
\end{align*}

Note that these formulations assume $\alpha >0$. When operationalizing, the sign of the price coefficient , $\alpha$, is negative, so we need to flip the sign of the elasticities.

For computing the elasticites, we use the results from the nonlinear GMM estimator, rather than the linear GMM estimator in Statsmodels. The nonlinear price coefficients are more consistent across years compared to the statsmodels results

In [ ]:
#sort the 1990 data by quantity
CarDataNL_1990 = CarDataNL_1990.sort_values(by=['Quantity'], ascending=False)
#create a 10x10 matrix of zeros
Elast_NL_1990 = np.zeros((10,10))
#On the diagonal, we calculate the price elasticity of demand for each observation
for i in range(10):
    Elast_NL_1990[i,i] = beta_1990[1]/(1-beta_1990[5]) * CarDataNL_1990['Price_Norm'].iloc[i]*(1- beta_1990[5]*CarDataNL_1990['Nest_Share'].iloc[i] - (1-beta_1990[5])*CarDataNL_1990['Market_Share'].iloc[i])
#On the off-diagonal, we calculate the cross-price elasticity of demand for each observation
for i in range(10):
    for j in range(10):
        if i != j:
            if CarDataNL_1990['Nests'].iloc[i] == CarDataNL_1990['Nests'].iloc[j]:
                Elast_NL_1990[i,j] = -beta_1990[1]* (CarDataNL_1990['Market_Share'].iloc[j]*CarDataNL_1990['Price_Norm'].iloc[i]/CarDataNL_1990['Market_Share'].iloc[i])*(CarDataNL_1990['Market_Share'].iloc[i] + (beta_1990[5]/(1-beta_1990[5]))*CarDataNL_1990['Nest_Share'].iloc[i])
            else:
                Elast_NL_1990[i,j] = -beta_1990[1]* CarDataNL_1990['Market_Share'].iloc[j]*CarDataNL_1990['Price_Norm'].iloc[j]
            
#print the matrix as a table
pd.DataFrame(Elast_NL_1990)

,0,1,2,3,4,5,6,7,8,9
0,-4.073890,0.006433,0.003633,0.141169,0.139149,0.004275,0.111580,0.004305,0.003317,0.098838
1,0.008234,-4.227703,0.209781,0.003475,0.005357,0.197024,0.003243,0.155165,0.153034,0.003463
2,0.008234,0.133487,-2.542015,0.003475,0.005357,0.118105,0.003243,0.093013,0.091735,0.003463
3,0.124382,0.006433,0.003633,-2.522532,0.084852,0.004275,0.068041,0.004305,0.003317,0.060271
4,0.194539,0.006433,0.003633,0.134639,-3.947278,0.004275,0.106419,0.004305,0.003317,0.094266
5,0.008234,0.167277,0.157585,0.003475,0.005357,-3.195069,0.003243,0.116558,0.114957,0.003463
6,0.146861,0.006433,0.003633,0.101642,0.100187,0.004275,-2.999724,0.004305,0.003317,0.071163
7,0.008234,0.213875,0.201483,0.003475,0.005357,0.189230,0.003243,-4.125316,0.146980,0.003463
8,0.008234,0.167065,0.157386,0.003475,0.005357,0.147815,0.003243,0.116410,-3.224034,0.003463
9,0.177012,0.006433,0.003633,0.122509,0.120756,0.004275,0.096832,0.004305,0.003317,-3.626637


In [ ]:
#repeat for 1991
CarDataNL_1991 = CarDataNL_1991.sort_values(by=['Quantity'], ascending=False)
Elast_NL_1991 = np.zeros((10,10))

for i in range(10):
    Elast_NL_1991[i,i] = beta_1991[1]/(1-beta_1991[5]) * CarDataNL_1991['Price_Norm'].iloc[i]*(1- beta_1991[5]*CarDataNL_1991['Nest_Share'].iloc[i] - (1-beta_1991[5])*CarDataNL_1991['Market_Share'].iloc[i])

for i in range(10):
    for j in range(10):
        if i != j:
            if CarDataNL_1991['Nests'].iloc[i] == CarDataNL_1991['Nests'].iloc[j]:
                Elast_NL_1991[i,j] = -beta_1991[1]* (CarDataNL_1991['Market_Share'].iloc[j]*CarDataNL_1991['Price_Norm'].iloc[i]/CarDataNL_1991['Market_Share'].iloc[i])*(CarDataNL_1991['Market_Share'].iloc[i] + (beta_1991[5]/(1-beta_1991[5]))*CarDataNL_1991['Nest_Share'].iloc[i])
            else:
                Elast_NL_1991[i,j] = -beta_1991[1]* CarDataNL_1991['Market_Share'].iloc[j]*CarDataNL_1991['Price_Norm'].iloc[j]

pd.DataFrame(Elast_NL_1991)

,0,1,2,3,4,5,6,7,8,9
0,-3.630980,0.005323,0.003420,0.127339,0.125532,0.003363,0.100616,0.004126,0.002849,0.089143
1,0.007173,-3.576953,0.184692,0.002706,0.005480,0.173563,0.003073,0.136710,0.134771,0.003114
2,0.007173,0.133596,-2.446244,0.002706,0.005480,0.118325,0.003073,0.093201,0.091879,0.003114
3,0.100361,0.005323,0.003420,-2.010913,0.068453,0.003363,0.054866,0.004126,0.002849,0.048610
4,0.206118,0.005323,0.003420,0.142611,-4.131989,0.003363,0.112683,0.004126,0.002849,0.099834
5,0.007173,0.139798,0.131758,0.002706,0.005480,-2.567754,0.003073,0.097528,0.096145,0.003114
6,0.144234,0.005323,0.003420,0.099794,0.098379,0.003363,-2.910947,0.004126,0.002849,0.069860
7,0.007173,0.217703,0.205183,0.002706,0.005480,0.192819,0.003073,-4.039622,0.149723,0.003114
8,0.007173,0.152513,0.143742,0.002706,0.005480,0.135080,0.003073,0.106399,-2.831486,0.003114
9,0.164943,0.005323,0.003420,0.114122,0.112504,0.003363,0.090173,0.004126,0.002849,-3.339176


In [ ]:
#repeat the above for 1992
CarDataNL_1992 = CarDataNL_1992.sort_values(by=['Quantity'], ascending=False)
Elast_NL_1992 = np.zeros((10,10))

for i in range(10):
    Elast_NL_1992[i,i] = beta_1992[1]/(1-beta_1992[5]) * CarDataNL_1992['Price_Norm'].iloc[i]*(1- beta_1992[5]*CarDataNL_1992['Nest_Share'].iloc[i] - (1-beta_1992[5])*CarDataNL_1992['Market_Share'].iloc[i])

for i in range(10):
    for j in range(10):
        if i != j:
            if CarDataNL_1992['Nests'].iloc[i] == CarDataNL_1992['Nests'].iloc[j]:
                Elast_NL_1992[i,j] = -beta_1992[1]* (CarDataNL_1992['Market_Share'].iloc[j]*CarDataNL_1992['Price_Norm'].iloc[i]/CarDataNL_1992['Market_Share'].iloc[i])*(CarDataNL_1992['Market_Share'].iloc[i] + (beta_1992[5]/(1-beta_1992[5]))*CarDataNL_1992['Nest_Share'].iloc[i])
            else:
                Elast_NL_1992[i,j] = -beta_1992[1]* CarDataNL_1992['Market_Share'].iloc[j]*CarDataNL_1992['Price_Norm'].iloc[j]

pd.DataFrame(Elast_NL_1992)

,0,1,2,3,4,5,6,7,8,9
0,-4.316308,0.175604,0.004355,0.005451,0.003243,0.144169,0.138642,0.004689,0.115334,0.003013
1,0.111160,-2.288738,0.004355,0.005451,0.003243,0.075818,0.072911,0.004689,0.060654,0.003013
2,0.007615,0.003327,-3.068701,0.167941,0.158129,0.005484,0.003742,0.140551,0.010389,0.115380
3,0.007615,0.003327,0.237169,-4.093788,0.210236,0.005484,0.003742,0.186865,0.010389,0.153400
4,0.007615,0.003327,0.149852,0.141078,-2.594852,0.005484,0.003742,0.118069,0.010389,0.096924
5,0.223172,0.185406,0.004355,0.005451,0.003243,-4.628201,0.146381,0.004689,0.121772,0.003013
6,0.158387,0.131584,0.004355,0.005451,0.003243,0.108029,-3.288806,0.004689,0.086422,0.003013
7,0.007615,0.003327,0.243784,0.229508,0.216099,0.005484,0.003742,-4.245390,0.010389,0.157678
8,0.528519,0.439080,0.004355,0.005451,0.003243,0.360481,0.346660,0.004689,-11.032648,0.003013
9,0.007615,0.003327,0.190814,0.179641,0.169145,0.005484,0.003742,0.150342,0.010389,-3.349873


(d) If we pool all three years' worth of data to estimate nested-logit, we are implicitly assuming that the market size can be explained by one of the two scenarios. One, the market size stays the same (100 million) and a given household can choose between cars of the same make/model that are released in different years, as well as cars of different makes/models that are released in the same year; this scenario captures intertemporal substitution of cars of the same make/model. The data assigns the same car model in different years to the same nest, imposing that consumers have 
the same taste for car models across different years, and the same substitution pattern of the same car model in different years.
Two, the market size increases to 300 million; this scenario assumes that 100 million new households enter the market each year and each household can only substitute between cars of different makes/models that are released in the same year. 

Q8. (a) The Multinomial Probit model is flexible, but there is a dimensionality issue because there are more than $J=5$ products in this market, which means there are potentially $J^2$ parameters to estimate; we will need a lot of computing power to simulate an integral of dimension $J$, which makes the estimation less feasible. The nested logit model is more appropriate in this setting because there are natural nests in the automobile market, and it is reasonable to assume that goods across nests have zero taste correlation. For example, if I am a consumer who wants to buy a truck, it is likely that I am only choosing a product from the nest of trucks and not from the nest of sports convertible.

(b) The Pure Characteristics model is well-suited in a market where quality (vertical characteristic) is dominant. The automobile market is horizontally differentiated in many dimensions, so the nested logit model, or a random coefficient logit model is more suited to model this setting.


PyBLP Tutorial

We begin by importing the package and reading in the data (Note: Make sure to import the packages in the first python cell of the notebook).

In [35]:
import pyblp
pyblp.options.digits = 2

product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
product_data.head()

ModuleNotFoundError: No module named 'pyblp'

Next, we specify the formulations of product characteristics.

In [ ]:
X1_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)') #these are the linear characteristics for the demand side
X2_formulation = pyblp.Formulation('1 + prices + sugar + mushy') #these are the nonlinear characteristics for the demand side
product_formulations = (X1_formulation, X2_formulation) #we combine the two formulations into one tuple
product_formulations #we print the tuple

(prices + Absorb[C(product_ids)], 1 + prices + sugar + mushy)

Now, we define an integration method for simulating market shares. Here we use Monte Carlo integration.

In [ ]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})
mc_integration

Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 0}.

Having defined our product formulation and integration method, we can now combine these into the problem class that PyBLP solves:

In [ ]:
mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
mc_problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD    ED 
---  ----  ---  ----  ----  ----  ----  ----
94   2256   5   4700   1     4     20    1  

Formulations:
       Column Indices:           0       1       2      3  
-----------------------------  ------  ------  -----  -----
 X1: Linear Characteristics    prices                      
X2: Nonlinear Characteristics    1     prices  sugar  mushy


Dimensions:
 T    N     F    I     K1    K2    MD    ED 
---  ----  ---  ----  ----  ----  ----  ----
94   2256   5   4700   1     4     20    1  

Formulations:
       Column Indices:           0       1       2      3  
-----------------------------  ------  ------  -----  -----
 X1: Linear Characteristics    prices                      
X2: Nonlinear Characteristics    1     prices  sugar  mushy

To solve the problem quickly, we use a solver with a loose tolerance and no support for parameter bounds.

In [ ]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})
bfgs

Configured to optimize using the BFGS algorithm implemented in SciPy with analytic gradients and options {gtol: +1.0E-04}.

We now estimate the model using the problem class and optimizer we just created. 

In [ ]:
results1 = mc_problem.solve(sigma=np.ones((4, 4)), optimization=bfgs)
results1

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     1       prices    sugar     mushy    |  Sigma Squared:     1       prices    sugar     mushy  
------  --------  --------  --------  --------  |  --------------  --------  --------  --------  --------
  1     +1.0E+00                                |        1         +1.0E+00  +1.0E+00  +1.0E+00  +1.0E+00
prices  +1.0E+00  +1.0E+00                      |      prices      +1.0E+00  +2.0E+00  +2.0E+00  +2.0E+00
sugar   +1.0E+00  +1.0E+00  +1.0E+00            |      sugar       +1.0E+00  +2.0E+00  +3.0E+00  +3.0E+00
mushy   +1.0E+00  +1.0E+00  +1.0E+00  +1.0E+00  |      mushy       +1.0E+00  +2.0E+00  +3.0E+00  +4.0E+00
Starting optimization ...



GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective   Gradient                                                                                                    
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares     Value    Improvement    Norm                                                  Theta                                               
----  -----------  ------------  -----------  -----------  -----------  -------  ---------  -----------  --------  --------------------------------------------------------------------------------------------------
 1     00:00:01         0             1          2898         8803         0     +7.5E+03                +7.3E+03  +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00, +1.0E+00
 1     00:00:01         0             2          1917         5860         0     +2.1E+03    +5.4E+03    +2.3E+03  +9.1E-01, +9.9E-01, +9.9E-01,

Problem Results Summary:
GMM   Objective  Gradient      Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step    Value      Norm    Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  ---------  --------  --------------  --------------  -------  ----------------  -----------------
 2    +1.5E+02   +8.7E-05     +8.5E-02        +6.5E+03        0         +5.2E+07          +8.3E+05     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:12       Yes          58           75          83992       258145   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1         prices      sugar       mushy     |  Sigma Squared:      1         prices      sugar       mushy   
------  ----------  ----------  ----------  ---